In [43]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [44]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [45]:
data_train = data_train.dropna()
data_test = data_test.dropna()

In [46]:
possible_labels = data_train.sentiment.unique()

In [47]:
#making a dictionary of different sentiments
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [48]:
label_dict

{'negative': 1, 'neutral': 0, 'positive': 2}

In [49]:
# cnverting sentiments into labels and assigning a new column to the labels
data_train['label'] = data_train.sentiment.replace(label_dict)
data_test['label'] = data_test.sentiment.replace(label_dict)

In [50]:
data_train.head()

,textID,text,selected_text,sentiment,label
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,0
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,1
2,088c60f138,my boss is bullying me...,bullying me,negative,1
3,9642c003ef,what interview! leave me alone,leave me alone,negative,1
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,1


In [51]:
data_test.head()

,textID,text,sentiment,label
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,2
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,1
3,01082688c6,happy bday!,positive,2
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,2


In [52]:
data_train.sentiment.value_counts()

neutral     11117
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [53]:
X_train = data_train['text'].values
y_train = data_train['label'].values
X_test = data_test['text'].values
y_test= data_test['label'].values

In [54]:
## loading tokenizer and encoding data

In [55]:
!pip install transformers

In [56]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [57]:
#using the pretrained uncased bert model with everything in lower case
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [58]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2) 

In [59]:
#encoding training and validation data
encoding_train = tokenizer.batch_encode_plus(
    X_train,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length = True,
    max_length=256,
    return_tensors='pt'
)

encoding_val = tokenizer.batch_encode_plus(
    X_val,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length = True,
    max_length=256,
    return_tensors='pt'
)

encoding_test = tokenizer.batch_encode_plus(
    X_test,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length = True,
    max_length=256,
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [60]:
X_train.shape, encoding_train.input_ids.shape

((21984,), torch.Size([21984, 256]))

In [61]:
X_test.shape, encoding_test.input_ids.shape

((3534,), torch.Size([3534, 256]))

In [62]:
X_val.shape, encoding_val.input_ids.shape

((5496,), torch.Size([5496, 256]))

## Converting data into bert format

In [63]:
input_ids_train = encoding_train['input_ids']
attention_masks_train = encoding_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoding_val['input_ids']
attention_masks_val = encoding_val['attention_mask']
labels_val = torch.tensor(y_val)

input_ids_test = encoding_test['input_ids']
attention_masks_test = encoding_test['attention_mask']
labels_test = torch.tensor(y_test)

In [64]:
training_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
validation_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_val)
testing_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

## setting up bert model

In [66]:
from transformers import  BertForSequenceClassification

In [67]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 3,
    output_attentions=False,
    output_hidden_states=False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## creating data loaders

In [69]:
from torch.utils.data import  DataLoader, RandomSampler, SequentialSampler

In [70]:
training_dataloader = DataLoader(
    training_dataset,
    sampler = RandomSampler(training_dataset),
    batch_size=32
)

validation_dataloader = DataLoader(
    validation_dataset,
    sampler = RandomSampler(validation_dataset),
    batch_size=32
)

testing_dataloader = DataLoader(
    testing_dataset,
    sampler = RandomSampler(testing_dataset),
    batch_size=32
)

## setting up optimizer and scheduler

In [72]:
from transformers import  AdamW, get_linear_schedule_with_warmup

In [73]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

In [74]:
epochs=5

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(training_dataloader)*epochs
)

## performance metrics

In [76]:
import numpy as np
from sklearn.metrics import f1_score

In [77]:
def calculate_f1_score(predictions, true_values):
  predictions = np.argmax(predictions, axis=1).flatten()
  true_values = true_values.flatten()
  return f1_score(true_values, predictions, average='weighted')

In [78]:
def accuracy_per_class(preds, labels):
  label_dict_inverse = {v: k for k, v in label_dict.items()}

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')

## training

In [80]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [82]:
if torch.cuda.is_available():
   model.cuda()

In [83]:
device

device(type='cuda')

In [84]:
def evaluate(dataloader):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [85]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(training_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }      

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(training_dataloader)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(validation_dataloader)
    val_f1 = calculate_f1_score(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.6523463973311878
Validation loss: 0.5308340244688267
F1 Score (Weighted): 0.7848899047272707



Epoch 2
Training loss: 0.4704841543449932
Validation loss: 0.5224424409138602
F1 Score (Weighted): 0.7945120482581668



Epoch 3
Training loss: 0.392048267558152
Validation loss: 0.5415279705857121
F1 Score (Weighted): 0.7940293939128646



Epoch 4
Training loss: 0.33847053737272514
Validation loss: 0.5643269404942213
F1 Score (Weighted): 0.79510119210561



Epoch 5
Training loss: 0.2989056964812928
Validation loss: 0.5943216011794501
F1 Score (Weighted): 0.7942434025326275



In [86]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [99]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_2.model', map_location=torch.device('cpu')))

<All keys matched successfully>

## predictions

In [100]:
_, predictions, true_vals = evaluate(testing_dataloader)

In [101]:
accuracy_per_class(predictions, true_vals)

Class: neutral
Accuracy: 1084/1430
Class: negative
Accuracy: 792/1001
Class: positive
Accuracy: 918/1103


In [102]:
predictions[2]

array([ 0.8932205,  1.5840336, -2.3841293], dtype=float32)

In [103]:
true_vals[2]

0

In [104]:
true_vals

array([2, 1, 0, ..., 0, 0, 1])

In [105]:
predictions_arg_max = []
for pred in predictions:
  predictions_arg_max.append(np.argmax(pred))

In [106]:
from sklearn.metrics import accuracy_score


In [107]:
print("Accuracy of BERT is:",accuracy_score(true_vals,predictions_arg_max))


Accuracy of BERT is: 0.790605546123373
